In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--lang=vi")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

In [2]:
url = 'https://www.google.com/maps/place/University+Medical+Center/@10.7552399,106.6644707,17z/data=!3m1!4b1!4m6!3m5!1s0x31752efa87dc6d6d:0x4feb0a1e755b6d62!8m2!3d10.7552399!4d106.6644707!16s%2Fg%2F11g6mdmpzh'
driver.get(url)

In [3]:
driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]').click()
#to make sure content is fully loaded we can use time.sleep() after navigating to each page
import time

time.sleep(1)

In [ ]:
total_number_of_reviews = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[2]/div/div[2]/div[2]').text.split(" ")[0]
print(total_number_of_reviews)
total_number_of_reviews = int(total_number_of_reviews.replace(',','')) if ',' in total_number_of_reviews else int(total_number_of_reviews)

#Find scroll layout
scrollable_div = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]')
#Scroll as many times as necessary to load all reviews
for _ in range(0, total_number_of_reviews, 10):
    count = 10
    while count > 0:
        try:
            driver.find_element_by_xpath("//button[contains(@aria-label,'More')]").click()
            time.sleep(2)
        except Exception as e:
            pass
        count -= 1
        
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
    time.sleep(20)

1,771


In [ ]:
from bs4 import BeautifulSoup

def get_review_summary(result_set):
    rev_dict = {
        'Review Time': [],
        'Review Text' : [],
        'Review Rate': [],}
    for result in result_set:

        review_rate = result.find('span', class_='kvMYJc')["aria-label"]
        review_time = result.find('span', class_="rsqaWe").text.strip()
        review_text = result.find('span',class_='wiI7pd')
    
        if review_text is None:
            rev_dict['Review Text'].append('')
        else:
            rev_dict['Review Text'].append(review_text.text.strip())
        
        rev_dict["Review Time"].append(review_time)
        
        rev_dict['Review Rate'].append(review_rate)
    return rev_dict

In [ ]:
import pandas as pd
response = BeautifulSoup(driver.page_source, 'html.parser')
reviews = response.find_all('div', class_='jftiEf')
rev_dict = get_review_summary(reviews)

In [ ]:
print(len(rev_dict["Review Text"]))

In [ ]:
df = pd.DataFrame.from_dict(rev_dict)
df.columns = ["Time", "Review Text", "Rating"]

In [ ]:
df.to_csv("DHYD-review.csv")

In [ ]:
pd.read_csv("DHYD-review.csv", index_col="Unnamed: 0")